# RadioWare Lab 3 - Antennas and Radio Propagation

## Purpose

The purpose of this lab is to build and measure two antennas, one for transmit and one for receive, for a specified BF / RF frequency (Baseband Frequency / Radio Frequency).

## Outline
- Using the soldered wire/SMA antenna (soldered by TAs), move to the FieldFox and measure reflection coefficient. Observe that the resonant frequency is significantly higher than baseband but not at a desirable frequency (i.e., 2.4 GHz ISM band).
- Progressively trim the antenna wires until the dipole antenna resonates at 2.4 GHz.
- Designate one antenna as the transmit antenna, and connect it to a VCO.
- Designate the other antenna as the receive antenna, and connect it to the scope or FieldFox.
- Verify the polarization effect.
- Take measurements of this "system" and connect to equations presented in lecture on the frequency response of the antennas as well as path loss.
- Save antennas in your lab kit box for upcoming labs.

## "Build it"

### Antenna fabrication
- Build two matched dipole antennas:
- Solder a 50mm wire to the center pin of an SMA connector and another 50mm wire to one of the ground pins/shield (Done by TA).
- Using S11 measurement in FieldFox Network Analyzer mode begin to trim the wires until the antenna becomes resonant (|S11|<-10dB) at 2.4 GHz.

### Validate link budgets

Baseband attempt:
- Connect the ADALM baseband directly to the transmit antenna and connect the receive antenna to the FieldFox. *Attempt* to measure the received power with the FieldFox at a close range such as 1 centimeter.
- Compute the effective gain of your antenna at baseband (it should be quite low)

RF attempt:
- Do the same thing at 2.4 GHz and validate your prelab link budget.
- Calibrate the VCO in your lab kit (Done by TA).
- Use your VCO to excite the antenna

### Friis experiments
Now, using the 2.4 GHz with your antennas, experiment with the various variables in the Friis equation
- Range: receive power decrease 6dB per doubling (octave) of the range and 20dB per decade of the range.
- Polarization: rotate your RX antenna wrt TX antenna at co-pol, cross-pol (90 degrees) and at 45 degrees and confirm the polarization efficiency of cos(theta) --- thus the 45 degree should be 70.7%.

## Getting started with antennas

### Background

The ISM radio bands are portions of the radio spectrum reserved internationally for industrial, scientific and medical (ISM) purposes. In the lab we are going to use 2.4-2.5 GHz radio frequency within the ISM bands. ISM contains other frequency bands. For detailed information about ISM band, please refer to https://en.wikipedia.org/wiki/ISM_band

We will build a half wavelength dipole antenna. For 2.4GHz radio, half wavelength is 62.5mm. A pair of antennas will be handed out to you. The half wavelength has been marked on the antennas.


#### Exercise 3.1: Antenna trimming
##### Antennas
A pair of antennas will be handed out to you. The TA soldered a 50mm wire to the center pin of an SMA connector and another 50mm wire to one of the ground pins/shield. The trimmed antennas look as follows.

<img src="https://ndwireless.github.io/radioware/lab3_01.png" width="40%">

##### S11
The S11 is a metric that measures the reflective power at the input port (of the antenna). We want the power reflected by the antenna to be small at our desired frequency. In other words, we want to make the antennas radiate at our desired frequency 2.4GHz, that is, |S11| < -10dB at 2.4GHz. 

S11 is measured with the FieldFox in network analyzer mode. Please connect the antenna to FieldFox port one (the left port). FieldFox should be on network analyzer mode (done by TA). The following is an example of an S11 measurement. As we can see, the trimmed antenna resonates at 2.4GHz. The less power reflected back, the more power radiated.

<img src="https://ndwireless.github.io/radioware/lab3_02.png" width="40%">
<br>

#### Antenna trimming
When the antenna assigned to you is first connected to the FieldFox, you may observe the resonant frequency is significantly higher than baseband but not at our desired frequency. You should trim the antenna to make it resonate at 2.4GHz.  

When you start trimming the antenna, only cut outside the halfwavelength marker. Better longer than shorter. If you trim too much, you will not be able to decrease the resonant frequency again.

By observing the S11 measurement, progressively trim the antenna wires symmetrically on both sides until the dipole antenna resonates at 2.4GHz. These antennas will be saved for this and upcoming labs.

##### Report
Please include final plots from the FieldFox for each antenna. What was the final length of each element of your dipole?

## Baseband Measurements

#### Exercise 3.2: Using ADALM2000 as baseband signal generator, measure the output signal power

##### Setup

Connect the modules according to the block diagram below. In this setup we use an SMA cable to connect the ADALM (W1 and GND) with the FieldFox. The center pin of the SMA cable is positive and the shield is ground. In the wiring picture, you can see a new interface board connected with the ADALM. This interface board facilitates the connection of the ADALM and a Voltage Controlled Oscillator (VCO). You can directly connect W1 on the board to the FieldFox. Note that we don't use the VCO in this exercise, but we will use it in the following exercises. Please ignore the VCO/amplifier connection in the wiring picture. They will be explained in the RF measurements.

##### Block Diagram
<img src="https://ndwireless.github.io/radioware/lab3_04.png" width="30%">

##### Wiring
<img src="https://ndwireless.github.io/radioware/lab3_12.png" width="50%">
Note that we do not power on the amplifier or VCO in this setup.

##### Generating a sine wave

After making the hardware connection shown above (i.e., connecting the W1 port to the FieldFox via a SMA cable), please run the following program. It is similar to the sine wave generation program as in Lab 1, except that: sampling rate (fs) 7.5MS/s and signal frequency (f0) 25KHz.

In [ ]:
import libm2k
import numpy as np
import time

fs = 7.5e06      # output(DAC) sampling rate in HZ
Ts = 1.0 / fs   # sampling period in seconds
f0 = 25e03     # sinusoidal wave frequency in HZ
T0 = 1.0 / f0   # period of the sinusoidal wave in seconds

uri = libm2k.getAllContexts()
if(len(uri) == 0):
    libm2k.contextCloseAll()

ctx=libm2k.m2kOpen()
ctx.calibrateDAC()
aout=ctx.getAnalogOut()

aout.setSampleRate(0, fs)
aout.enableChannel(0, True)

t=np.arange(0,T0,Ts)
buffer0 = np.sin(2*np.pi*f0*t)

aout.setCyclic(True)
aout.push(0,buffer0) # Push the buffer to channel 0 of the output


##### Reading data from FieldFox and print on local machine
Run the FieldFox data read program below to measure the signal power. It is a similar program as in Lab 1 except that the settings have been changed to: center frequency ($120$ KHz), bandwidth ($200$ KHz), resolution bandwidth ($1$ KHz), number of average traces ($1$), y reference ($10$ dBm). 

In [ ]:
# sample code which works for Keysight FieldFox N9914B
import visa
import sys
import numpy as np
import matplotlib.pyplot as plt
import time

rm = visa.ResourceManager('C:\\Windows\\System32\\visa64.dll')
try:
    VISA_ADDRESS = "USB0::0x2A8D::0x5C18::MY59221093::0::INSTR"  # set the resource string for your instrument
    inst = rm.open_resource(VISA_ADDRESS)  # open the connection
except visa.Error as ex:
    print('Couldn\'t connect to \'%s\', exiting now...' % VISA_ADDRESS)
    sys.exit()
inst.InputBufferSize = 1000000 # set the buffer size
inst.ByteOrder = 'littleEndian' # set the Byte order
inst.timeout = 10000 # set the timeout value

# Now control the instrument using SCPI commands.
# Refer to the instrument programming manual for your instrument for the correct SCPI commands for your instrument.

span = 0.2 #MHz, bandwidth
bdw = 1 #KHz, resolution bandwidth
currRFFreq = 0.12 #MHz, center frequency
y_ref = 10 #dBm, y reference
ave_number = 1 #number of average traces

inst.write('SYST:PRES;*OPC?')#preset the instrument and wait for operation complete via the *OPC?
a = inst.read()
inst.write("INST:SEL 'SA';*OPC?")
a = inst.read()
inst.write('FREQ:SPAN ' + str(span) + ' MHz')#Set Span
inst.write('BAND ' + str(bdw) + ' KHz') #Set RES BW
inst.write('FREQ:CENT ' + str(currRFFreq) + ' MHz') #Set Center Frequency
time.sleep(1)
inst.write('DISPlay:WINDow:TRACe:Y:RLEVel ' + str(y_ref))#set y axis reference level
inst.write('SENS:SWE:POIN?')#query number of points in trace
numPoints = int(inst.read())
inst.write('SENS:FREQ:START?')#query start frequency
startFreq = float(inst.read())
inst.write('SENS:FREQ:STOP?')#query stop frequency
stopFreq = float(inst.read())
x = np.linspace(startFreq, stopFreq, numPoints)
data_array = np.zeros((1, numPoints))

inst.write('TRAC1:TYPE AVG')
inst.write('SENSe:AVERage:TYPE AUTO')#set average mode
inst.write('SENSe:AVERage:COUNt ' + str(ave_number))#set average count

inst.write('INIT:CONT OFF;*OPC?')#set trigger mode to hold for trigger synchronization
a = inst.read()
inst.write('INIT:IMM;*OPC?')#assert a single trigger and wait for trigger complete
a = inst.read()
inst.write('INIT:REST;*OPC?')
a = inst.read()

inst.write('TRAC1:DATA?')#query the response data
data = inst.read()
data_array = np.array(data.split(',')).astype('float64')#use split to turn long string to an array of values
print('The peak power is ' + str(max(data_array)) + 'dBm')
average_power = 10*np.log10(np.mean(np.power(10, data_array/10)))
print('The average power is ' + str(average_power) + 'dBm')

inst.write('INIT:CONT ON;*OPC?')
a = inst.read()
# Close the VISA connection
inst.close()
rm.close()
# Plot the spectrum
plt.figure(figsize=(15, 10))
plt.xlabel('Freq(Hz)')
plt.ylabel('Spectrum(dBm)')
plt.title('Average of '+ str(ave_number)+' traces spectrum (Span ' + str(span) + ' MHz ' + ' RBW ' + str(bdw) + 'KHz)')
plt.plot(x, data_array)
plt.savefig('fieldfox.png')
plt.show()

Below is an example of the spectrum of the generated signal.

<img src="https://ndwireless.github.io/radioware/lab3_05.png" width="50%">


Example result

'The peak power is 3.9989498dBm'

Note: We ignore the loss of the cable between the ADALM and the FieldFox in the baseband measurements because the loss is negligible at this low frequency.

Please rename and save your picture (fieldfox.png). Otherwise it will be overwritten when you do the following experiments.

##### Report
Please include final plot from the FieldFox.

Calculate the signal power the ADALM generates and compare with the measurement (ADALM output impedance is 50 Ohm and the FieldFox input impedance is also 50 Ohm). 

#### Exercise 3.3: Using ADALM2000 as baseband signal generator and the handmade antennas, measure the received signal power

In this exercise, we will replace the wireline link (SMA cable) with a wireless link.

##### Setup
Use the handmade antennas instead of a wireline link between the ADALM and the FieldFox. Set the distance of the transmit antenna and the receiver antenna to be 1 cm.

###### Setup diagram
<img src="https://ndwireless.github.io/radioware/lab3_07.png" width="40%">

###### Wiring
<img src="https://ndwireless.github.io/radioware/lab3_13.png" width="60%">

The TX antenna is the one directly connected to W1 port of the interface board and the RX antenna is the one connected to the FieldFox.


##### Use the ADALM to generate a sine wave and read data 
Rerun the two programs in Exercise 3.2 to print out the power of the signal received by FieldFox. What do you observe compared to Exercise 3.2?

Note: Rename and save your picture (fieldfox.png) before you rerun the programs; otherwise, it will replace the existing picture. 

Example result

'The peak power is -97.1001205dBm'

<img src="https://ndwireless.github.io/radioware/lab3_08.png" width="50%">

We can even increase the signal amplitude. Try to set sine wave amplitude as to be 2 in Exercise 3.2 and rerun the programs. Can you observe the received signal at the FieldFox? 


##### Report
Please include the plot from the FieldFox.

Question: What conclusion can you draw regarding the effective gain of your antenna at baseband?

Hint: Based on the Friis transmission equation and antenna gain equations, develop a link budget for a small (62.5mm) dipole with 25 KHz inputs over a range of 1 centimeter. Leave the gain of the baseband antenna as an unknown. You can compute the gain by the power measurements.

## RF measurements: 

#### Exercise 3.4: Using VCO (+amplifier+attenuator) as Radio Frequency signal generator, measure the generated signal power

In this exercise, we add the VCO as a Radio Frequency (RF) signal generator but we will focus on output power as in Exercise 3.2. The VCO acts as the Local Oscillator (LO) in the transmitter and in the receiver of the communication system. The frequency of the signal generated by the VCO corresponds to the voltage we apply across VCO Vtune pin and Ground. The ADALM provides an adjustable power supply(-5V to 5V). In our labs we use the ADALM to control VCO Vtune and generate signals of different frequencies (2.0GHz - 2.6 GHz).

The amplifier and the attenuator are mainly used as an isolator between the VCO and the mixer which will be added in the upcoming lab. According to their datasheets, the VCO has $4$ dBm output power, the amplifier has $14$ dB gain and the attenuator has $10$ dB loss. Therefore we expect an $8$ dBm signal from the attenuator output.

##### Setup
Connect the modules according to the block diagram below. All the connections in the block diagram are implemented on the interface board. You only need to connect the attenuator output with the FieldFox via an SMA cable. Then power on the VCO and amplifier (plug in the two pigtail power cables). Note that the picture does not show the "ground plane" board that should be used to support the additional units connected to the adaptor board.

###### Setup diagram
<img src="https://ndwireless.github.io/radioware/lab3_09.png" width="40%">

###### Wiring
<img src="https://ndwireless.github.io/radioware/lab3_14.png" width="60%">
<br>

##### Generate LO by VCO
For each set of VCO, amplifier and attenuator, a calibration table will be provided to you by the TA. A number may be found on the backside of the amplifier. You may find its calibration table within the calibration tables folder. For example, if the number on the amplifier is 3, you can find its calibration table VCO03.png within the calibration tables folder. 

In the table, the x axis is Vtune and the y axis is its generated signal frequency. There are two curves in the table. 'datasheet curve' is for the single VCO obtained from its datasheet. You should refer to 'measured curve' to control your VCO. Save this table for upcoming labs.

Look up the table and find the value of voltage needed to set Vtune that generates a 2.4 GHz signal. Use this value to set Vtune in the following program, and run the program.

#### ADALM2000 as an adjustable power supply (serving as VCO Vtune)

In [ ]:
#sample code to set dc voltage when ADALM2000 works as an adjustable power supply
import libm2k


vtune = 3.26 #please change this value to generate the desired frequency


uri = libm2k.getAllContexts()
if(len(uri) == 0):
    libm2k.contextCloseAll()
#open and calibrate ADALM2000
ctx=libm2k.m2kOpen()
ctx.calibrateDAC()
ps = ctx.getPowerSupply()
ps.enableChannel(0, True)

#Use ADALM2000 to generate VCO Vtune
ps.pushChannel(0, vtune)

##### Check generated signal and measure its power with the FieldFox
Run the FieldFox data read program in Exercise 3.2 and check the frequency of the generated signal. Please set center frequency ($2400$ MHz), bandwidth ($20$ MHz), resolution bandwidth ($100$ KHz), number of average traces ($1$), y reference ($10$ dBm). 

Example result

'The peak power is 8.1315136dBm'

Note 1: You may need to fine tune Vtune to get close to $2.4$ GHz, i.e., slightly change vtune and rerun the ADALM power supply program until the desired frequency is obtained. The Vtune sensitivity is about 130MHz/V. You do not need an exact $2.4$ GHz. From your S11 plot obtained in Exercise 3.1, the closer to the resonant frequency, the higher the antenna gain will be.

Note 2: The VCO generated signal frequency varies within a $100$ KHz frequency range. Using an average of more than one trace will decrease the measured power.

Note 3: The loss of the cable connecting to the FieldFox is 1dB at 2.4GHz. Take this into account in the calculation. 

Note 4: If you cannot obtain the desired signal, use V+ and Ground test points on the interface board to verify that a correct Vtune value is applied. The oscilloscope DVM can be used to measure the DC voltage.

##### Report
Please include the final plot from the FieldFox and a description of your tuning process.

What is the measured RF signal generator (VCO/AMP/attenuator) output power?

#### Exercise 3.5: Using the VCO (amplifier, attenuator) as a RF signal generator and the handmade antennas, measure the received signal power

In this exercise, we will add the antennas and observe the effects of distance and orientation.

##### Setup
Use handmade antennas instead of wireline link between attenuator and FieldFox. 

###### Setup diagram
<img src="https://ndwireless.github.io/radioware/lab3_10.png" width="40%">

###### Wiring
<img src="https://ndwireless.github.io/radioware/lab3_15.png" width="50%">

##### Distance
Set the distance of the transmit antenna and the receiver antenna to be $1$ dm (10 cm), $2$ dm, and $1$ m and rerun programs in Exercise 3.4 with the same parameters and print out the power of the signal received by FieldFox for each distance. 

Example result

d = 1 dm 'The peak power is -9.2800579dBm' (Assuming the antennas have 0 dB gain, according to Friis equation, expected path loss is 20.22 dB. Measured path loss -9.28 - 8.13 = -17.41dB)

d = 2 dm 'The peak power is -15.5117311dBm' (Expected 6 dB decrease from 1 dm measurement, measured -15.51 + 9.28 = -6.23dB)

d = 1 m 'The peak power is -28.9745731dBm' (Expected 20 dB decrease from 1 dm measurement, measured -28.97 + 9.28 = -19.69dB)

##### Report
Please include the final plots from the FieldFox.

Question: What do you expect and what do you observe for different distances? 

Hint: Based on the Friis transmission equation and antenna gain equations, develop a link budget for a small (62.5mm) dipole with 2.4 GHz input over different ranges. Use lecture equations for the antenna gain at 2.4 GHz.

##### Polarization of Antennas
Now we will fix the distance between the two antennas to be 1 dm and change the orientation of the two antennas to be 0 degree, 45 degree, and 90 degree. Rerun programs in Exercise 3.4 with the same parameters and print out the power of the signal received by the FieldFox for each angle. 

##### Report
Question: What do you expect and what do you observe for each different orientation?

Example result

d = 1 dm, orientation = 45 degree 'The peak power is -12.6182985dBm'

d = 1 dm, orientation = 90 degree 'The peak power is -32.8938637dBm'